In [3]:
from Main import clean_save_csv, optimal_clusters

#csv_file_path = clean_save_csv()
#best_k_elbow, best_k_silhouette, best_k_combined = optimal_clusters(csv_file_path, max_clusters=20, alpha=0.6, plot=True)